In [ ]:
import os
import random
import glob
import warnings
import pickle
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, SubsetRandomSampler
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

from time import time
from PIL import Image
import math


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
torch.manual_seed(4460);
np.random.seed(4460)

In [ ]:
import torch
from torch import nn, cuda, device, optim
print('currentdevice=',torch.cuda.current_device())
print('totaldevice=',torch.cuda.device_count())
print('changedevice=',torch.cuda.device(1))
print('availabledevice=',torch.cuda.is_available())

device = device('cuda:0')
print(device)


In [ ]:
# Dataset folder used
DATASET_PATH = os.path.join('./AATTCT-IDS/Image/Extracted/')

# We would like to perform a train-validation-test split at the ratio of T:V:T = 6:2:2.
VAL_SPLIT = 0.3
TEST_SPLIT = 0.1
img_size = size=256
# Batch size for training. Limited by GPU memory
BATCH_SIZE = 2
# Training Epochs
epochs = 100

In [ ]:
class SELayer(nn.Module):
    def __init__(self, channel, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(channel, channel // reduction),
                nn.ReLU(inplace=True),
                nn.Linear(channel // reduction, channel),        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        y = torch.clamp(y, 0, 1)
        return x * y

def depthwise_conv(inp, oup, kernel_size=3, stride=1, relu=False):
    return nn.Sequential(
        nn.Conv2d(inp, oup, kernel_size, stride, kernel_size//2, groups=inp, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU(inplace=True) if relu else nn.Sequential(),
    )

class GhostModule(nn.Module):
    def __init__(self, inp, oup, kernel_size=1, ratio=2, dw_size=3, stride=1, relu=True):
        super(GhostModule, self).__init__()
        self.oup = oup
        init_channels = math.ceil(oup / ratio)
        new_channels = init_channels*(ratio-1)

        self.primary_conv = nn.Sequential(
            nn.Conv2d(inp, init_channels, kernel_size, stride, kernel_size//2, bias=False),
            nn.BatchNorm2d(init_channels),
            nn.ReLU(inplace=True) if relu else nn.Sequential(),
        )

        self.cheap_operation = nn.Sequential(
            nn.Conv2d(init_channels, new_channels, dw_size, 1, dw_size//2, groups=init_channels, bias=False),
            nn.BatchNorm2d(new_channels),
            nn.ReLU(inplace=True) if relu else nn.Sequential(),
        )

    def forward(self, x):
        x1 = self.primary_conv(x)
        x2 = self.cheap_operation(x1)
        out = torch.cat([x1,x2], dim=1)
        return out[:,:self.oup,:,:]


class GhostBottleneck(nn.Module):
    def __init__(self, inp, hidden_dim, oup, kernel_size, stride):
        super(GhostBottleneck, self).__init__()
        assert stride in [1, 2]

        self.conv = nn.Sequential(
            # pw
            GhostModule(inp, hidden_dim, kernel_size=1, relu=True),
            # dw
            depthwise_conv(hidden_dim, hidden_dim, kernel_size, stride, relu=False) if stride==2 else nn.Sequential(),
            # Squeeze-and-Excite
            SELayer(hidden_dim),
            # pw-linear
            GhostModule(hidden_dim, oup, kernel_size=1, relu=False),
        )

        if stride == 1 and inp == oup:
            self.shortcut = nn.Sequential()
        else:
            self.shortcut = nn.Sequential(
                depthwise_conv(inp, inp, kernel_size, stride, relu=False),
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )

    def forward(self, x):
        return self.conv(x) + self.shortcut(x)

#Nested Unet

class Ghost_Unet(nn.Module):
    """
    Implementation of this paper:
    https://arxiv.org/pdf/1807.10165.pdf
    """
    def __init__(self, in_ch=1, out_ch=1):
        super(Ghost_Unet, self).__init__()

        n1 = 16
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.GBneck0_0 = GhostBottleneck(in_ch, filters[0], filters[0],3,1)
        self.GBneck1_0 = GhostBottleneck(filters[0], filters[1], filters[1],3,1)
        self.GBneck2_0 = GhostBottleneck(filters[1], filters[2], filters[2],3,1)
        self.GBneck3_0 = GhostBottleneck(filters[2], filters[3], filters[3],3,1)
        self.GBneck4_0 = GhostBottleneck(filters[3], filters[4],filters[4],3,1)

        self.GBneck0_1 = GhostBottleneck(filters[0] + filters[1], filters[0], filters[0],3,1)
        self.GBneck1_1 = GhostBottleneck(filters[1] + filters[2], filters[1], filters[1],3,1)
        self.GBneck2_1 = GhostBottleneck(filters[2] + filters[3], filters[2], filters[2],3,1)
        self.GBneck3_1 = GhostBottleneck(filters[3] + filters[4], filters[3], filters[3],3,1)

        self.GBneck0_2 = GhostBottleneck(filters[0]*2 + filters[1], filters[0], filters[0],3,1)
        self.GBneck1_2 = GhostBottleneck(filters[1]*2 + filters[2], filters[1], filters[1],3,1)
        self.GBneck2_2 = GhostBottleneck(filters[2]*2 + filters[3], filters[2], filters[2],3,1)

        self.GBneck0_3 = GhostBottleneck(filters[0]*3 + filters[1], filters[0], filters[0],3,1)
        self.GBneck1_3 = GhostBottleneck(filters[1]*3 + filters[2], filters[1], filters[1],3,1)

        self.GBneck0_4 = GhostBottleneck(filters[0]*4 + filters[1], filters[0], filters[0],3,1)

        self.final = nn.Conv2d(filters[0], out_ch, kernel_size=1)

    def forward(self, x):
        
        x0_0 = self.GBneck0_0(x)
        x1_0 = self.GBneck1_0(self.pool(x0_0))
        x0_1 = self.GBneck0_1(torch.cat([x0_0, self.Up(x1_0)], 1))

        x2_0 = self.GBneck2_0(self.pool(x1_0))
        x1_1 = self.GBneck1_1(torch.cat([x1_0, self.Up(x2_0)], 1))
        x0_2 = self.GBneck0_2(torch.cat([x0_0, x0_1, self.Up(x1_1)], 1))

        x3_0 = self.GBneck3_0(self.pool(x2_0))
        x2_1 = self.GBneck2_1(torch.cat([x2_0, self.Up(x3_0)], 1))
        x1_2 = self.GBneck1_2(torch.cat([x1_0, x1_1, self.Up(x2_1)], 1))
        x0_3 = self.GBneck0_3(torch.cat([x0_0, x0_1, x0_2, self.Up(x1_2)], 1))

        x4_0 = self.GBneck4_0(self.pool(x3_0))
        x3_1 = self.GBneck3_1(torch.cat([x3_0, self.Up(x4_0)], 1))
        x2_2 = self.GBneck2_2(torch.cat([x2_0, x2_1, self.Up(x3_1)], 1))
        x1_3 = self.GBneck1_3(torch.cat([x1_0, x1_1, x1_2, self.Up(x2_2)], 1))
        x0_4 = self.GBneck0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.Up(x1_3)], 1))

        output = self.final(x0_4)
        output = F.sigmoid(output)
        return output

In [ ]:
def dice_coefficient(predicted, target):
   
        #     Returns:
    #         coefficient(float): Dice coefficient for the input sample.
        #                                     1 represents highest similarity and

    # The smooth term is used to prevent division by zero.
    smooth = 1
    product = np.multiply(predicted, target)
    intersection = np.sum(product)
    coefficient = (2 * intersection + smooth) / (np.sum(predicted) + np.sum(target) + smooth)
    return coefficient

In [ ]:
unet_model = None
unet_classifier = None
#criterion = nn.BCELoss()
learning_rate = 0.0001

#### If you want to see the training trend within each epoch, you can change mini_batch to a positive integer 
#### that is no larger than the number of batches per epoch.
mini_batch = False
#FILTER_LIST = [16,32,64,128,256]
# Define where to save the model parameters.
model_save_path = './saved_models/'
os.makedirs(model_save_path, exist_ok = True)

# New model is created.
unet_model = Ghost_Unet().to(device)

#### You can uncomment this to see the textual architecture of our U-Net.
print(unet_model)

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

In [ ]:
def dice_loss(inputs, targets):
        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = F.sigmoid(inputs)       
        smooth=1
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

In [ ]:
def iou_score(output, target, smooth=1e-6):
    output = torch.sigmoid(output)
    output = (output > 0.5).float()
    intersection = (output * target).sum()
    union = output.sum() + target.sum() - intersection
    iou = (intersection + smooth) / (union + smooth)
    return iou

In [ ]:
from sklearn.model_selection import KFold
k_folds = 10
# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

In [ ]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(unet_model)

In [ ]:
Training_Validation_Data=(train_indices+validation_indices)

In [ ]:
%%time
start=time()
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np
import os
from tqdm import tqdm  # For progress bar
# Define the paths for images and masks
image_dir = './AATTCT-IDS/Image/Extracted/'
mask_dir = './AATTCT-IDS/Label/Visceral/'

# Initialize the dataset with the correct arguments
custom_dataset = CustomDataset(image_dir=image_dir, mask_dir=mask_dir, img_size=256)
for fold,(train_idx,val_idx) in enumerate(kfold.split(Training_Validation_Data)):
    
    print('------------fold no---------{}----------------------'.format(fold))
    # Get indices for train, validation, and test split
    train_indices, val_indices, test_indices = get_indices(len(custom_dataset), val_split=VAL_SPLIT, test_split=TEST_SPLIT)
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)
    # Create DataLoaders
    train_loader = DataLoader(custom_dataset, batch_size=BATCH_SIZE, sampler=train_sampler, shuffle=False)
    val_loader = DataLoader(custom_dataset, batch_size=1, sampler=val_sampler, shuffle=False)

    #train_sampler, validation_sampler = SubsetRandomSampler(train_idx), SubsetRandomSampler(test_idx)
    #trainloader = torch.utils.data.DataLoader(CustomDataset(DATASET_PATH), BATCH_SIZE, sampler = train_idx)
    #validationloader = torch.utils.data.DataLoader(CustomDataset(DATASET_PATH), 1, sampler = val_idx)
    
        # Training session history data.
    history = {'train_loss': list(), 'validation_loss': list(), 'train_score': list(), 'validation_score': list()}

    # For save best feature. Initial loss taken a very high value.
    last_score = 0

    # Optimizer used for training process. Adam Optimizer.
    optimizer = optim.Adam(unet_model.parameters(), lr = learning_rate)
    # Reducing LR on plateau feature to improve training.
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.85, patience = 2, verbose = True)
    early_stopping = EarlyStopping(patience=15, verbose=True)
    print('Starting Training Process')

    assert validationloader.batch_size == 1

    # Epoch Loop
    for epoch in range(epochs):

        #################################### Train ####################################################
        unet_model.train()
        start_time = time()
        # Training a single epoch
        train_epoch_loss, train_batch_loss, batch_iteration = 0, 0, 0
        validation_score, validation_loss = 0, 0
        Training_score=0
        Training_iou_score = 0
        for batch, data in enumerate(trainloader):
            # Keeping track how many iteration is happening.
            batch_iteration += 1
            # Loading data to device used.
            image = data['image'].to(device)
            mask = data['mask'].to(device)
            # Clearing gradients of optimizer.
            optimizer.zero_grad()
            # Calculation predicted output using forward pass.
            output = unet_model(image)
            # Calculating the loss value.
            loss_value = dice_loss(output, mask)
            # Computing the gradients.
            loss_value.backward()
            # Optimizing the network parameters.
            optimizer.step()
            # Updating the running training loss
            train_epoch_loss += loss_value.item()
            train_batch_loss += loss_value.item()

            # Threshold elimination.
            output = (output > 0.5)
            output = output.cpu().numpy()
            mask = mask.cpu().numpy()
            mask = np.resize(mask, (1, size, size))
            output = np.resize(output, (1, size, size))
            # Calculate the dice score for original and predicted image mask.
            Training_score += dice_coefficient(output, mask)
            # Calculate the IoU score for original and predicted image mask.
            Training_iou_score += iou_score(torch.tensor(output), torch.tensor(mask))

            # Printing batch logs if any. Useful if you want to see the training trends within each epoch.
            if mini_batch:
                if (batch + 1) % mini_batch == 0:
                    train_batch_loss = train_batch_loss / (mini_batch * trainloader.batch_size)
                    print(
                        f'    Batch: {batch + 1:2d},	Batch Loss: {train_batch_loss:.7f}')
                    train_batch_loss = 0

        train_epoch_loss = train_epoch_loss / (batch_iteration * trainloader.batch_size)
        unet_train = Training_score / batch_iteration
        unet_train_iou = Training_iou_score / batch_iteration
        ################################### Validation ##################################################
        unet_model.eval()
        # To get data in loops.
        batch_iteration = 0

        validation_iou_score = 0
        for batch, data in enumerate(validationloader):
            # Keeping track how many iteration is happening.
            batch_iteration += 1
            # Data prepared to be given as input to model.
            image = data['image'].to(device)
            mask = data['mask'].to(device)
           # Predicted output from the input sample.
            mask_prediction = unet_model(image)

            # comput validation loss
            loss_value = dice_loss(mask_prediction, mask)
            validation_loss += loss_value.item()

            # Threshold elimination.
            mask_prediction = (mask_prediction > 0.5)
            mask_prediction = mask_prediction.cpu().numpy()
            mask = mask.cpu().numpy()

            mask = np.resize(mask, (1, size, size))
            mask_prediction = np.resize(mask_prediction, (1, size, size))
            # Calculate the dice score for original and predicted image mask.
            validation_score += dice_coefficient(mask_prediction, mask)
            # Calculate the IoU score for original and predicted image mask.
            validation_iou_score += iou_score(torch.tensor(mask_prediction), torch.tensor(mask))

        # Calculating the mean score for the whole validation dataset.
        unet_val = validation_score / batch_iteration
        unet_val_iou = validation_iou_score / batch_iteration
        validation_loss = validation_loss / batch_iteration

        # Collecting all epoch loss mse = mean_squared_error(predicted, target)
        history['train_loss'].append(train_epoch_loss)
        history['validation_loss'].append(validation_loss)

        # Collecting all epoch score values for future visualization.
        history['train_score'].append(unet_train)
        history['validation_score'].append(unet_val)

        # Reduce LR On Plateau
        scheduler.step(validation_loss)

        time_taken = time() - start_time

        # Training Logs printed.
        print(f'Epoch: {epoch + 1:3d},  ', end = '')
        print(f'train Loss: {train_epoch_loss:.5f},  ', end = '')
        print(f'Training score (Dice): {unet_train:.5f},  ', end = '')
        print(f'Training score (IoU): {unet_train_iou:.5f},  ', end = '')
        print(f'validation Loss: {validation_loss:.5f},  ', end = '')
        print(f'validation score (Dice): {unet_val:.5f},  ', end = '')
        print(f'validation score (IoU): {unet_val_iou:.5f},  ', end = '')

        for pg in optimizer.param_groups:
            print('current lr: ', pg['lr'], ', ', end = '')
        print(f'Time: {time_taken:.2f} s', end = '')

        early_stopping(validation_loss, unet_model)

        if early_stopping.early_stop:
            print("Early stopping")
            break
        # Save the model every epoch.
        current_epoch_model_save_path = os.path.join(model_save_path, 'GhostUnet_Scapis_Abdomen_VAT_epoch_%s.pth' % (str(epoch).zfill(3)))
        torch.save(unet_model.state_dict(), current_epoch_model_save_path)

        # Save the best model (determined by validation score) and give it a unique name.
        best_model_path = os.path.join(model_save_path, 'GhostUnet_Scapis_Abdomen_VAT_best_model.pth')
        if  last_score < unet_val:
            torch.save(unet_model.state_dict(), best_model_path)
            last_score = unet_val
            print(f'	Best model saved at score: {unet_val:.5f}')
        else:
            print()

    print(f'Training Finished after {epochs} epoches')
    
# end = time.time ()
# time spent = (end-start) /60
# print (f" (time spent: .3} minutes") 
print(f'Time: {time() - start}')


In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(15,10))
plt.grid(color='r', linestyle='dotted', linewidth=0.5)
plt.plot(history['train_loss'], 'o-', color = '#9900CC')
plt.plot(history['validation_loss'], 'o-', color = '#00cc33')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
# from matplotlib import pyplot as plt
plt.figure(figsize=(15,10))
plt.grid(color='r', linestyle='dotted', linewidth=0.5)
plt.plot(history['train_score'], 'o-', color = '#9900CC')
plt.plot(history['validation_score'], 'o-', color = '#00cc33')
plt.title('model score')
plt.ylabel('Dice score')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
%%time
start=time()
# Load the unet model at its prime (when it performed the best on the validation set).
state_dict = torch.load(os.path.join(model_save_path, 'GhostUnet_Scapis_Abdomen_VAT_best_model.pth'))
unet_model.load_state_dict(state_dict)

# Testing process on test data.
unet_model.eval()
# Getting test data indices for dataloading
test_data_indexes = test_indices
# Total testing data used.
data_length = len(test_data_indexes)
# Score after testing on dataset.
mean_test_dice_score = 0
mean_test_iou_score = 0

for batch, data in enumerate(testloader):
    # Data prepared to be given as input to model.
    image = data['image'].to(device)
    mask = data['mask']

    # Predicted output from the input sample.
    mask_prediction = unet_model(image).cpu()
    # Threshold elimination.
    mask_prediction = (mask_prediction > 0.5)
    mask_prediction = mask_prediction.numpy()

    mask = np.resize(mask, (1, img_size, img_size))
    mask_prediction = np.resize(mask_prediction, (1, img_size, img_size))

    # Calculating the dice score for original and predicted mask.
    mean_test_dice_score += dice_coefficient(mask_prediction, mask)
    # Calculating the IoU score for original and predicted mask.
    mean_test_iou_score += iou_score(torch.tensor(mask_prediction), torch.tensor(mask))

# Calculating the mean scores for the whole test dataset.
unet_dice_score = mean_test_dice_score / data_length
unet_iou_score = mean_test_iou_score / data_length
# Putting the model back to training mode.
print(f'\nDice Score {unet_dice_score}\n')
print(f'IoU Score {unet_iou_score}\n')

print(f'Time: {time() - start}')

In [ ]:
from skimage import segmentation
#result_image = segmentation.mark_boundaries(input_image, segmentation_results, mode='thick')
from scipy import ndimage
from skimage import measure

In [ ]:
def result(image, mask, output, title, transparency = 0.02, save_path = None):
    '''
    Plots a 2x3 plot with comparisons of output and original image.
    Works best with Jupyter Notebook/Lab.
    Parameters:
        image(numpy.ndarray): Array containing the ofrom skimage import segmentation
#result_image = segmentation.mark_boundaries(input_image, segmentation_results, mode='thick')

from scipy import ndimageriginal image of MRI scan.
        mask(numpy.ndarray): Array containing the original mask of tumor.
        output(numpy.ndarray): Model predicted mask from input image.
        title(str): Title of the plot to be used.
        transparency(float): Transparency level of mask on images.
                             Default: 0.38
        save_path(str): Saves the plot to the location specified.
                        Does nothing if None. 
                        Default: None
    Return:
        None
    '''

    fig, axs = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(
        20, 15), gridspec_kw={'wspace': 0.025, 'hspace': 0.010})
    fig.suptitle(title, x=0.5, y=0.92, fontsize=20)

    axs[0][0].set_title("Original Mask", fontdict={'fontsize': 16})
    axs[0][0].imshow(mask, cmap='gray')
    axs[0][0].set_axis_off()

    axs[0][1].set_title("Predicted Mask", fontdict={'fontsize': 16})
    axs[0][1].imshow(output, cmap='gray')
    axs[0][1].set_axis_off()

    mask_diff = np.abs(np.subtract(mask, output))
    axs[0][2].set_title("Mask Difference", fontdict={'fontsize': 16})
    axs[0][2].imshow(mask_diff, cmap='gray')
    axs[0][2].set_axis_off()

    seg_output = mask*transparency
    seg_image = np.add(image, seg_output)/2
    axs[1][0].set_title("Original Segmentation", fontdict={'fontsize': 16})
    axs[1][0].imshow(seg_image, cmap='gray')
    axs[1][0].set_axis_off()

    seg_output = output*transparency

    labeled_seg_output, _ = ndimage.label(seg_output)
    #result_image = segmentation.mark_boundaries(image, labeled_seg_output, mode='thick')
    seg_image = np.add(image, labeled_seg_output)/2
    result_image = segmentation.mark_boundaries(seg_image, output,color=(1, 1, 0), mode='thick')
    
    axs[1][1].set_title("Predicted Segmentation", fontdict={'fontsize': 16})
    axs[1][1].imshow(result_image, cmap='gray')
    axs[1][1].set_axis_off()

    axs[1][2].set_title("Original Input Image", fontdict={'fontsize': 16})
    axs[1][2].imshow(image, cmap='gray')
    axs[1][2].set_axis_off()

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi = 90, bbox_inches = 'tight')

    plt.show()

In [ ]:
%%time
start=time()
for example_index in range(10):
    # The purpose of image_index is to make sure we truly pick from the test set.
    
    image_index = test_indices[example_index]
    sample = custom_dataset[image_index]
    threshold = 0.5

    unet_model.eval()
    image = sample['image'].numpy()
    mask = sample['mask'].numpy()

    image_tensor = torch.Tensor(image)
    image_tensor = image_tensor.view((-1, 1, img_size, img_size)).to(device)
    output = unet_model(image_tensor).detach().cpu()
    output = (output > threshold)
    output = output.numpy()

    # image(numpy.ndarray): 512x512 Original brain scanned image.
    image = np.resize(image, (img_size, img_size))
    # mask(numpy.ndarray): 512x512 Original mask of scanned image.
    mask = np.resize(mask, (img_size, img_size))
    # output(numpy.ndarray): 512x512 Generated mask of scanned image.                                            
    output = np.resize(output, (img_size, img_size))
    # score(float): Sørensen–Dice Coefficient for mask and output. Calculates how similar are the two images.
    d_score = dice_coefficient(output, mask)
    # score(float): IoU for mask and output.
    iou = iou_score(torch.tensor(output), torch.tensor(mask))

    title = f'Name: {image_index}.png   Dice Score: {d_score:.5f}   IoU Score: {iou:.5f}'
    save_path = os.path.join('./output_ghostnet/',f'{d_score:.5f}_{image_index}.png')
    result(image, mask, output, title, save_path = None)
